<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Laura", "transactions": [{"transaction-id": 56, "amount": 829}, {"transaction-id": 116, "amount": 818}, {"transaction-id": 369, "amount": 822}, {"transaction-id": 801, "amount": 781}, {"transaction-id": 833, "amount": 758}, {"transaction-id": 1000, "amount": 756}, {"transaction-id": 1207, "amount": 761}, {"transaction-id": 1645, "amount": 819}, {"transaction-id": 2137, "amount": 772}, {"transaction-id": 2244, "amount": 772}, {"transaction-id": 2456, "amount": 851}, {"transaction-id": 2649, "amount": 789}, {"transaction-id": 2896, "amount": 807}, {"transaction-id": 2923, "amount": 753}, {"transaction-id": 2978, "amount": 795}, {"transaction-id": 3091, "amount": 801}, {"transaction-id": 3104, "amount": 763}, {"transaction-id": 3166, "amount": 759}, {"transaction-id": 3466, "amount": 807}, {"transaction-id": 3682, "amount": 835}, {"transaction-id": 3877, "amount": 785}, {"transaction-id": 4220, "amount": 812}, {"transaction-id": 4564, "amount": 839}, {"transaction-id"

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Laura", "transactions": [{"transaction-id": 56, "amount": 829}, {"transaction-id": 116, "amount": 818}, {"transaction-id": 369, "amount": 822}, {"transaction-id": 801, "amount": 781}, {"transaction-id": 833, "amount": 758}, {"transaction-id": 1000, "amount": 756}, {"transaction-id": 1207, "amount": 761}, {"transaction-id": 1645, "amount": 819}, {"transaction-id": 2137, "amount": 772}, {"transaction-id": 2244, "amount": 772}, {"transaction-id": 2456, "amount": 851}, {"transaction-id": 2649, "amount": 789}, {"transaction-id": 2896, "amount": 807}, {"transaction-id": 2923, "amount": 753}, {"transaction-id": 2978, "amount": 795}, {"transaction-id": 3091, "amount": 801}, {"transaction-id": 3104, "amount": 763}, {"transaction-id": 3166, "amount": 759}, {"transaction-id": 3466, "amount": 807}, {"transaction-id": 3682, "amount": 835}, {"transaction-id": 3877, "amount": 785}, {"transaction-id": 4220, "amount": 812}, {"transaction-id": 4564, "amount": 839}, {"transaction-id"

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Laura',
  'transactions': [{'transaction-id': 56, 'amount': 829},
   {'transaction-id': 116, 'amount': 818},
   {'transaction-id': 369, 'amount': 822},
   {'transaction-id': 801, 'amount': 781},
   {'transaction-id': 833, 'amount': 758},
   {'transaction-id': 1000, 'amount': 756},
   {'transaction-id': 1207, 'amount': 761},
   {'transaction-id': 1645, 'amount': 819},
   {'transaction-id': 2137, 'amount': 772},
   {'transaction-id': 2244, 'amount': 772},
   {'transaction-id': 2456, 'amount': 851},
   {'transaction-id': 2649, 'amount': 789},
   {'transaction-id': 2896, 'amount': 807},
   {'transaction-id': 2923, 'amount': 753},
   {'transaction-id': 2978, 'amount': 795},
   {'transaction-id': 3091, 'amount': 801},
   {'transaction-id': 3104, 'amount': 763},
   {'transaction-id': 3166, 'amount': 759},
   {'transaction-id': 3466, 'amount': 807},
   {'transaction-id': 3682, 'amount': 835},
   {'transaction-id': 3877, 'amount': 785},
   {'transaction-id': 4220, 'amount'

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 25,
  'name': 'Alice',
  'transactions': [{'transaction-id': 87, 'amount': 17},
   {'transaction-id': 296, 'amount': 14},
   {'transaction-id': 334, 'amount': 17},
   {'transaction-id': 480, 'amount': 16},
   {'transaction-id': 591, 'amount': 15},
   {'transaction-id': 832, 'amount': 15},
   {'transaction-id': 892, 'amount': 17},
   {'transaction-id': 1194, 'amount': 18},
   {'transaction-id': 1346, 'amount': 16},
   {'transaction-id': 1627, 'amount': 17},
   {'transaction-id': 1743, 'amount': 15},
   {'transaction-id': 1942, 'amount': 16},
   {'transaction-id': 2605, 'amount': 14},
   {'transaction-id': 3703, 'amount': 16},
   {'transaction-id': 4115, 'amount': 17},
   {'transaction-id': 4437, 'amount': 18},
   {'transaction-id': 4465, 'amount': 14},
   {'transaction-id': 4596, 'amount': 17},
   {'transaction-id': 4647, 'amount': 17},
   {'transaction-id': 4744, 'amount': 16},
   {'transaction-id': 4846, 'amount': 16},
   {'transaction-id': 5191, 'amount': 16},
   {'transactio

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 61},
 {'name': 'Alice', 'count': 39},
 {'name': 'Alice', 'count': 7},
 {'name': 'Alice', 'count': 79},
 {'name': 'Alice', 'count': 54})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(61, 39, 7, 79, 54)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

51.353923205342234

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 87, 'amount': 17},
  {'transaction-id': 296, 'amount': 14},
  {'transaction-id': 334, 'amount': 17},
  {'transaction-id': 480, 'amount': 16},
  {'transaction-id': 591, 'amount': 15},
  {'transaction-id': 832, 'amount': 15},
  {'transaction-id': 892, 'amount': 17},
  {'transaction-id': 1194, 'amount': 18},
  {'transaction-id': 1346, 'amount': 16},
  {'transaction-id': 1627, 'amount': 17},
  {'transaction-id': 1743, 'amount': 15},
  {'transaction-id': 1942, 'amount': 16},
  {'transaction-id': 2605, 'amount': 14},
  {'transaction-id': 3703, 'amount': 16},
  {'transaction-id': 4115, 'amount': 17},
  {'transaction-id': 4437, 'amount': 18},
  {'transaction-id': 4465, 'amount': 14},
  {'transaction-id': 4596, 'amount': 17},
  {'transaction-id': 4647, 'amount': 17},
  {'transaction-id': 4744, 'amount': 16},
  {'transaction-id': 4846, 'amount': 16},
  {'transaction-id': 5191, 'amount': 16},
  {'transaction-id': 5328, 'amount': 16},
  {'transaction-id': 5436, 'amount': 15},


In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 87, 'amount': 17},
 {'transaction-id': 296, 'amount': 14},
 {'transaction-id': 334, 'amount': 17})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(17, 14, 17)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

2118.1977829069274

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 132), ('Alice', 132), ('Alice', 167), ('Alice', 168), ('Bob', 101), ('Bob', 129), ('Bob', 228), ('Charlie', 88), ('Charlie', 88), ('Charlie', 111), ('Charlie', 112), ('Dan', 88), ('Dan', 88), ('Dan', 112), ('Dan', 112), ('Edith', 128), ('Edith', 131), ('Edith', 163), ('Edith', 166), ('Frank', 53), ('Frank', 54), ('Frank', 68), ('Frank', 68), ('George', 109), ('George', 110), ('George', 140), ('George', 140), ('Hannah', 133), ('Hannah', 135), ('Hannah', 170), ('Hannah', 175), ('Ingrid', 77), ('Ingrid', 77), ('Ingrid', 98), ('Ingrid', 98), ('Jerry', 109), ('Jerry', 109), ('Jerry', 138), ('Jerry', 138), ('Kevin', 107), ('Kevin', 108), ('Kevin', 137), ('Kevin', 139), ('Laura', 121), ('Laura', 121), ('Laura', 153), ('Laura', 153), ('Michael', 99), ('Michael', 99), ('Michael', 126), ('Michael', 126), ('Norbert', 99), ('Norbert', 99), ('Norbert', 125), ('Norbert', 126), ('Oliver', 55), ('Oliver', 55), ('Oliver', 70), ('Oliver', 70), ('Patricia', 43), ('Patricia', 44), ('Patricia', 

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 599), ('Bob', 458), ('Charlie', 399), ('Dan', 400), ('Edith', 588), ('Frank', 243), ('George', 499), ('Hannah', 613), ('Ingrid', 350), ('Jerry', 494), ('Kevin', 491), ('Laura', 548), ('Michael', 450), ('Norbert', 449), ('Oliver', 250), ('Patricia', 199), ('Quinn', 686), ('Ray', 450), ('Sarah', 500), ('Tim', 400), ('Ursula', 489), ('Victor', 750), ('Wendy', 514), ('Xavier', 527), ('Yvonne', 300), ('Zelda', 650)]
CPU times: user 182 ms, sys: 9.15 ms, total: 191 ms
Wall time: 632 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Laura,"[{'transaction-id': 56, 'amount': 829}, {'tran..."
1,1,Kevin,"[{'transaction-id': 211, 'amount': 1970}, {'tr..."
2,2,Xavier,"[{'transaction-id': 99, 'amount': 249}, {'tran..."
3,3,Victor,"[{'transaction-id': 390, 'amount': 1588}, {'tr..."
4,4,Wendy,"[{'transaction-id': 1820, 'amount': 2760}, {'t..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 226 ms, sys: 22.3 ms, total: 249 ms
Wall time: 1.75 s


name
Alice      599
Bob        458
Charlie    399
Dan        400
Edith      588
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Laura', 'amount': 829, 'transaction-id': 56},
 {'id': 0, 'name': 'Laura', 'amount': 818, 'transaction-id': 116},
 {'id': 0, 'name': 'Laura', 'amount': 822, 'transaction-id': 369})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Laura,829,56
1,0,Laura,818,116
2,0,Laura,822,369
3,0,Laura,781,801
4,0,Laura,758,833


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 237 ms, sys: 17.2 ms, total: 254 ms
Wall time: 1.54 s


name
Alice       30761
Bob         23609
Charlie     14374
Dan         10755
Edith       17300
Frank        2731
George      17491
Hannah      26809
Ingrid       8833
Jerry       24056
Kevin       28192
Laura       18242
Michael     13012
Norbert     14154
Oliver       4669
Patricia     3772
Quinn       31890
Ray         12847
Sarah       31531
Tim         18266
Ursula      24150
Victor      38929
Wendy       18709
Xavier      35609
Yvonne      11878
Zelda       17431
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()